# Demo: Oncotools

### Import modules

In [1]:
from oncotools.connect import Database
from oncotools.data_elements.dose_map import DoseMask

import numpy as np

### Connect to the database

In [2]:
db = Database(db='OncospaceHeadNeck', us='oncoguest', pw='0ncosp@ceGuest')

### Patient ID and Patient representation ID

Individual patients are identified by a patientID. 

Each patient has at least one patient representations. A patient representation corresponds to one scan (a certain geometry) and a set of structures from that one scan.

In [3]:
# Map patient representation ID to patient ID
patLUT = db.patient_representations.get_patient_id_LUT()

# Map patient ID to patient representation ID(s)
prepLUT = db.patient_representations.get_patient_rep_id_LUT()

### Get a list of ROI's

Get a list of all the ROI's in the database

In [4]:
db.regions_of_interest.get_roi_names()

['2500_planning',
 '2cmring',
 '4cmring',
 '5000_planning',
 '5040_planning',
 '5400_planning',
 '5412_planning',
 '5425_planning',
 '5440_planning',
 '5520_planning',
 '5600_planning',
 '5760_planning',
 '5800_planning',
 '5810_planning',
 '5940_planning',
 '6000_planning',
 '6120_planning',
 '6125_planning',
 '6240_planning',
 '6290_planning',
 '6300_planning',
 '6400_planning',
 '6650_planning',
 '6720_planning',
 '6800_planning',
 '6960_planning',
 '6cmring',
 '7000_planning',
 '7040_planning',
 '7200_planning',
 '7208_planning',
 'ajw_ctv1_6000',
 'ajw_gtv',
 'ajw_new_ctv1',
 'ant_digastric',
 'ar_ctv1_tumor+bilateral_neck_nodes',
 'ar_gtv',
 'ar_l_i-v_neck_nodes',
 'ar_r_i-v_neck_nodes',
 'auto_r_parotid',
 'bone',
 'brachial_plexus',
 'brain',
 'brain-ptv',
 'brain2',
 'brainstem',
 'brainstem+5mm',
 'brainstem-5mm',
 'canal',
 'carina',
 'cerebellum',
 'chiasm',
 'comb_gtv',
 'comb_gtv_n',
 'comb_gtv_p',
 'comb_lung',
 'comb_parotids',
 'comb_preop_gtv',
 'combinded_submandibul

### Get a list of patients with certain ROIs

In [5]:
# Pick an ROI or a list of ROI's
rois = ['l_parotid', 'r_parotid']

In [6]:
# Get a list of patient representation's with all the ROI's we specified
patient_list = db.regions_of_interest.get_patient_rep_ids_with_rois(rois)
print len(patient_list), 'patient representations with given ROI\'s'

1034 patient representations with given ROI's


In [7]:
# Let's pick one of these patients
mypatient = patient_list[0]

# The major data types:

## Regions of Interest (ROIs)

Binary masks of an anatomical structure or a contour from a patient scan
The `Roi` class stores information for each ROI, and contains a `Mask` object.

The image is stored as a 3D matrix with binary values (1 = in the structure, 0 = not in the structure)

### Get ROI's or Masks for a set of structures

`db.regions_of_interest.get_rois()` and `db.regions_of_interest.get_masks()` return:
- a dictionary mapping ROI name to the Roi/Mask object
- a list of ROI's that were not found

In [8]:
# Get certain ROI's for our patient
myrois, _ = db.regions_of_interest.get_rois(mypatient, rois)
myrois

{'l_parotid': <oncotools.data_elements.roi.Roi at 0x69c5240>,
 'r_parotid': <oncotools.data_elements.roi.Roi at 0x72c4710>}

In [9]:
# We can also get just the masks
mymasks, _ = db.regions_of_interest.get_masks(mypatient, rois)
mymasks

{'l_parotid': <oncotools.data_elements.image.Mask at 0x72b7208>,
 'r_parotid': <oncotools.data_elements.image.Mask at 0x69c5198>}

In [10]:
# Let's pick a mask
amask = mymasks['l_parotid']

Masks are images with the following fields

In [11]:
print amask

Dimension:  3
Origin:     [-29.94140625, -87.5414047241211, 52.75]
End:        [29.941661335527897, -27.658337138593197, 93.84931570291519]
Spacing:    [0.1171879991889, 0.1171879991889, 0.2999950051307678]
Index:      [0.0, 0.0, 0.0]
Size:       (512L, 512L, 138L) (voxels)
Direction:  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]


They are represented as a 3D bitmap. Note that the X and Z coordinates are reversed in the data matrix, so be careful directly indexing the data matrix.

In [12]:
print amask.data.shape

(138L, 512L, 512L)


Masks can also be represented as a point cloud in physical coordinates

In [13]:
mymasks['l_parotid'].transform_to_point_cloud()

array([[  8.49625748, -61.17410491,  66.24977523],
       [  8.61344548, -61.17410491,  66.24977523],
       [  8.73063348, -61.17410491,  66.24977523],
       ..., 
       [  7.55875349, -59.88503692,  73.14966035],
       [  7.67594149, -59.88503692,  73.14966035],
       [  7.79312949, -59.88503692,  73.14966035]])

## Dose Grids

Dose grids store the dose that is delivered to a patient for treatment.

Image data is stored as a 3D matrix with `float` values for the dose at each voxel

### Get a dose grid

`db.regions_of_interest.get_rois()` returns a dictionary mapping the dose grid name to the `Dose` object

In [14]:
mydoses = db.radiotherapy_sessions.get_dose(mypatient)
mydoses

{'trial': <oncotools.data_elements.dose.Dose at 0x4b955f8>}

In [15]:
# Let's pick a dose grid
adose = mydoses['trial']

## Dose Masks

Dose masks map a dose grid onto an ROI. Since the dose grids and the ROI's are on different coordinate systems, this process involves a coordinate transformation and interpolation.

Image data is stored as a 3D matrix with `float` values for the dose at each voxel in the ROI

### Construct a dose mask object

Specify a mask and a dose grid to construct a `DoseMask`

In [16]:
dm = DoseMask(amask, adose)

In [17]:
print dm

Dimension:  3
Origin:     [-29.94140625 -87.54140472  52.75      ]
End:        [ 29.94166134 -27.65833714  93.8493157 ]
Spacing:    [ 0.117188    0.117188    0.29999501]
Index:      [ 0.  0.  0.]
Size:       [512 512 138] (voxels)
Direction:  [ 1.  0.  0.  0.  1.  0.]
Max Dose:   6641.13908131
Min Dose:   1379.36212036
Mean Dose:  3648.98874391
Std Dose:   1583.47037293



### Dose masks have DVH data

- Column 0: dose values in centigray (x-axis)
- Column 1: fraction of volume (y-axis)

In [18]:
dm.dvh_data

array([[  1.37936212e+03,   1.00000000e+00],
       [  1.40580321e+03,   9.99927468e-01],
       [  1.43224430e+03,   9.99637340e-01],
       [  1.45868539e+03,   9.99347211e-01],
       [  1.48512648e+03,   9.98912019e-01],
       [  1.51156757e+03,   9.97751505e-01],
       [  1.53800866e+03,   9.95865671e-01],
       [  1.56444975e+03,   9.93399579e-01],
       [  1.59089084e+03,   9.89337782e-01],
       [  1.61733193e+03,   9.83680279e-01],
       [  1.64377302e+03,   9.77152390e-01],
       [  1.67021411e+03,   9.69899180e-01],
       [  1.69665520e+03,   9.62718503e-01],
       [  1.72309629e+03,   9.54667440e-01],
       [  1.74953738e+03,   9.45455864e-01],
       [  1.77597847e+03,   9.35736563e-01],
       [  1.80241956e+03,   9.25074345e-01],
       [  1.82886065e+03,   9.14702256e-01],
       [  1.85530174e+03,   9.03169653e-01],
       [  1.88174284e+03,   8.92362370e-01],
       [  1.90818393e+03,   8.80684703e-01],
       [  1.93462502e+03,   8.69079568e-01],
       [  

### Extract certain points from a DVH

**DX parameters**

Get the dose to X percent of the structure. For example, D25 (a high dose) is the dose to 25% (0.25) of a structure

In [19]:
dm.get_dose_to_volume(0.25)

5319.0845685062313

**VX parameters**

Get the volume that received a certain dose. For example, V55 is the fraction of the structure that received 5500 cGy

In [20]:
dm.get_volume_with_dose(5500)

0.21469500253862336